In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from kneed import KneeLocator

import string
import re
import os

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
lemmatizer = nltk.stem.WordNetLemmatizer()
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sergejromanov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/sergejromanov/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [5]:
current_dir = os.getcwd()  # gets current working directory
project_dir = os.path.dirname(current_dir)  
directory_path = os.path.dirname(current_dir) + os.sep + 'data' + os.sep

In [6]:
#read the pkl file for movies
df = pd.read_pickle(directory_path + 'full_movies.pkl')
#read the pkl file for shows
df_2 = pd.read_pickle(directory_path + 'full_shows.pkl')

In [7]:
#read json file activity for movies
# activity_movie = pd.read_json('./movie_activities.json')
# activity_shows = pd.read_json('./show_activities.json')
activities = pd.read_json(directory_path + 'activities.json')

In [8]:
activities.head()
activities.groupby('user_id').count()

,content_id,activity,datetime
user_id,,,
0,426,426,426
1,406,406,406
2,386,386,386
3,414,414,414
4,420,420,420
5,233,233,233
6,219,219,219
7,227,227,227
8,227,227,227


## TF-IDF + kmeans clustering

In [9]:
stop_words = stopwords.words('english')
def clean_text(x):
    x = x.lower()
    x = x.translate(str.maketrans('', '', string.punctuation))
    # tokenize
    x = nltk.word_tokenize(x)
    x = [token for token in x if token not in stop_words]
    # lemmatization and pass string back
    x = ' '.join([lemmatizer.lemmatize(w) for w in x])
    return x

In [10]:
df['features'] = df['description'].apply(lambda x: clean_text(x))
df_2['features'] = df_2['description'].apply(lambda x: clean_text(x))

In [11]:
#tf-idf and kmeans for all movies
# Convert the text data to a matrix of TF-IDF features
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(df['features'])


inertia = []
#calculate the optimal number of clusters
K = range(1, 100)
for k in K:
    kmeanModel = KMeans(n_clusters=k,random_state=0)
    kmeanModel.fit(tfidf)
    inertia.append(kmeanModel.inertia_)


kn = KneeLocator(K, inertia, curve='convex', direction='decreasing')
print(kn.knee)


/Users/sergejromanov/miniconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


82


In [12]:
#tf-idf and kmeans for all tv-shows
# Convert the text data to a matrix of TF-IDF features
tfidf_vectorizer_shows = TfidfVectorizer(stop_words='english')
tfidf_shows = tfidf_vectorizer_shows.fit_transform(df_2['features'])


inertia = []
#calculate the optimal number of clusters
K = range(1, 20)
for k in K:
    kmeanModel = KMeans(n_clusters=k,random_state=0)
    kmeanModel.fit(tfidf_shows)
    inertia.append(kmeanModel.inertia_)


kn = KneeLocator(K, inertia, curve='convex', direction='decreasing')
print(kn.knee)

15


In [13]:
# Cluster the documents using KMeans
num_clusters = 13
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf)

# Assign cluster labels to each document
clusters = km.labels_.tolist()
df['cluster'] = clusters
df.head()

,episode_title,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,features,cluster
0,The Way We Were,0,Movies,The Way We Were,NaN,The Way We Were,A love story that begins with the attraction o...,A love story that begins with the attraction o...,"[ABC TV, ABC TV Plus, DRAMA, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/zy/ZY933...,[['Director Sydney Pollack']],"[abc1, abc2, drama, classic, film, feature-len...",2021-06-25 07:00:00,PG,6802.0,NaN,,love story begin attraction opposite katie hub...,3
1,Starman,1,Movies,Starman,NaN,Starman,An alien stranded on Earth clones himself into...,An alien stranded on Earth clones himself into...,"[ABC TV, ABC TV Plus, DRAMA, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW301...,[['Director John Carpenter']],"[abc1, abc2, drama, sci-fi, fantasy, romance, ...",2021-06-25 07:00:00,PG,6612.0,NaN,,alien stranded earth clone form young woman de...,12
2,Mr Deeds Goes To Town,2,Movies,Mr Deeds Goes To Town,NaN,Mr Deeds Goes To Town,A small-town poet inherits a vast fortune and ...,A small-town poet inherits a vast fortune and ...,"[ABC TV, ABC TV Plus, DRAMA, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW302...,"[['Cast Gary Cooper', 'Jean Arthur', 'Douglas ...","[abc1, abc2, drama, comedy-drama, film, featur...",2021-06-25 07:00:00,G,6649.0,NaN,,smalltown poet inherits vast fortune find new ...,6
3,On The Road,3,Movies,On The Road,NaN,On The Road,Aspiring writer Sal Paradise has his world roc...,Aspiring writer Sal Paradise has his world roc...,"[ABC TV, ABC TV Plus, DRAMA, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW291...,[['Director Walter Salles']],"[abc1, abc2, usa, drama, adaptation, adventure...",2021-06-01 07:00:00,MA,8014.0,NaN,,aspiring writer sal paradise world rocked foll...,11
4,The Boys Are Back,4,Movies,The Boys Are Back,NaN,The Boys Are Back,"Journalist Joe Warr and his two sons, from dif...","Journalist Joe Warr and his two sons, from dif...","[ABC TV, ABC TV Plus, DRAMA, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW238...,[['Director Scott Hicks']],"[abc1, abc2, drama, comedy-drama, film, single...",2021-06-11 07:00:00,M,5950.0,NaN,,journalist joe warr two son different marriage...,7


In [14]:
# Cluster the documents using KMeans
num_clusters_shows = 8
km_shows = KMeans(n_clusters=num_clusters_shows)
km_shows.fit(tfidf_shows)

# Assign cluster labels to each document
clusters_shows = km_shows.labels_.tolist()
df_2['cluster'] = clusters_shows
df_2.reset_index(inplace=True)
df_2.head()

/Users/sergejromanov/miniconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,index,id,title,category,description,description2,directors_actors,publication_date,rating,duration_sec,tags,tags2,image,features,cluster
0,0,10000,199 Little Heroes,Education,The journey to school is a very special type o...,"[Enjo lives in Quinten, a forest glade in the ...",[[]],2021-06-01 06:20:00,G,"[311.0, 311.0, 312.0, 311.0, 311.0, 320.0, 311...","[ABC ME, EDUCATION]","[primary-humanities, education, abc3, refugee-...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW073...,journey school special type life experience sh...,4
1,1,10001,7.30,News,Leigh Sales presents Australia's leading night...,[The devastation caused by the recent floods i...,[[]],2022-03-14 20:00:00,None,"[1906.0, 1894.0, 1879.0, 1935.0, 1891.0, 1822....","[ABC TV, ABC NEWS]","[lismore, news, childcare, flooding, abc-faves...",https://cdn.iview.abc.net.au/thumbs/i/nc/NC220...,leigh sale present australia leading nightly p...,0
2,2,10002,7.30 Mark Humphries Satire,News,Satirist Mark Humphries brings his unique pers...,[Satirist Mark Humphries goes inside the Healt...,[[]],2021-05-11 14:00:00,None,"[145.0, 110.0, 118.0, 136.0, 202.0, 115.0, 241...",[ABC NEWS],"[mark, parachuting, humphries, candidates, sus...",https://cdn.iview.abc.net.au/thumbs/i/nn/NN211...,satirist mark humphries brings unique perspect...,0
3,3,10003,7.30: The Interviews,News,Revisit some of 7.30's most compelling convers...,[Musician Dave Grohl sits down with 7.30's Lei...,[[]],2022-03-08 20:00:00,None,[1030.0],[ABC NEWS],"[news24, news]",https://cdn.iview.abc.net.au/thumbs/i/ns/NS224...,revisit 730s compelling conversation australia...,0
4,4,10004,72 Cutest Animals,Family,The animal kingdom continually astounds us wit...,[Cute is a notion that can mean many things to...,[[]],2022-03-05 15:03:06,G,"[1616.0, 1616.0, 1620.0, 1620.0, 1614.0, 1620....","[ABC ME, FAMILY]","[family-viewing, family-viewing-abcme, abc3]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW243...,animal kingdom continually astounds u showcase...,6


In [15]:
australian = 'Australia|australia|australian|Australian|Aboriginal|Indigenous|First People'
diverse = 'queer|lbgtqi|diversity|inclusion|disability|aboriginal|race|racism|indigenous|equal-opportunit'

In [16]:
expat_library = df[df['tags2'].str.contains(australian) | df['description'].str.contains(australian) | df['description2'].str.contains(australian) | df['title'].str.contains(australian)
]
expat_library_shows= df_2[df_2['tags2'].str.contains(australian) | df_2['description'].str.contains(australian) | df_2['description2'].str.contains(australian) | df_2['title'].str.contains(australian)
]
diversity_library = df[df['tags2'].str.contains(diverse) | df['description'].str.contains(diverse) | df['description2'].str.contains(diverse) | df['title'].str.contains(diverse)]
diversity_library_shows= df_2[df_2['tags2'].str.contains(diverse) | df_2['description'].str.contains(diverse) | df_2['description2'].str.contains(diverse) | df_2['title'].str.contains(diverse)
]

In [17]:
#reset index for df_2
df_2.reset_index(inplace=True)
#drop the column 'index'
df_2.drop(columns=['index'], inplace=True)
df_2.head()

,level_0,id,title,category,description,description2,directors_actors,publication_date,rating,duration_sec,tags,tags2,image,features,cluster
0,0,10000,199 Little Heroes,Education,The journey to school is a very special type o...,"[Enjo lives in Quinten, a forest glade in the ...",[[]],2021-06-01 06:20:00,G,"[311.0, 311.0, 312.0, 311.0, 311.0, 320.0, 311...","[ABC ME, EDUCATION]","[primary-humanities, education, abc3, refugee-...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW073...,journey school special type life experience sh...,4
1,1,10001,7.30,News,Leigh Sales presents Australia's leading night...,[The devastation caused by the recent floods i...,[[]],2022-03-14 20:00:00,None,"[1906.0, 1894.0, 1879.0, 1935.0, 1891.0, 1822....","[ABC TV, ABC NEWS]","[lismore, news, childcare, flooding, abc-faves...",https://cdn.iview.abc.net.au/thumbs/i/nc/NC220...,leigh sale present australia leading nightly p...,0
2,2,10002,7.30 Mark Humphries Satire,News,Satirist Mark Humphries brings his unique pers...,[Satirist Mark Humphries goes inside the Healt...,[[]],2021-05-11 14:00:00,None,"[145.0, 110.0, 118.0, 136.0, 202.0, 115.0, 241...",[ABC NEWS],"[mark, parachuting, humphries, candidates, sus...",https://cdn.iview.abc.net.au/thumbs/i/nn/NN211...,satirist mark humphries brings unique perspect...,0
3,3,10003,7.30: The Interviews,News,Revisit some of 7.30's most compelling convers...,[Musician Dave Grohl sits down with 7.30's Lei...,[[]],2022-03-08 20:00:00,None,[1030.0],[ABC NEWS],"[news24, news]",https://cdn.iview.abc.net.au/thumbs/i/ns/NS224...,revisit 730s compelling conversation australia...,0
4,4,10004,72 Cutest Animals,Family,The animal kingdom continually astounds us wit...,[Cute is a notion that can mean many things to...,[[]],2022-03-05 15:03:06,G,"[1616.0, 1616.0, 1620.0, 1620.0, 1614.0, 1620....","[ABC ME, FAMILY]","[family-viewing, family-viewing-abcme, abc3]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW243...,animal kingdom continually astounds u showcase...,6


In [18]:
# Print the top 10 terms for each cluster
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = tfidf_vectorizer.get_feature_names_out()
for i in range(num_clusters):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :13]:
        print(' %s' % terms[ind], end='')
    print()



Cluster 0: stage geography science filmed 45 live eruption including mumbai program impact film universe
Cluster 1: set life police tell story stick corruption man change tragedy person crime honour
Cluster 2: place year new dragon lunar work tradition race end chinese celebrate 2010 university
Cluster 3: young life turn soon love wash boy story hand family body decide like
Cluster 4: school play special episode new high family emma big join safe coronavirus help
Cluster 5: young book julia animated donaldson based 2020 world axel scheffler knew detective night
Cluster 6: story people disability mark film australian beach day short greatest international man tell
Cluster 7: world 2009 father 2007 based novel sea son travel life way begin secret
Cluster 8: performed ballet australian music orchestra present australia opera love victoria production time symphony
Cluster 9: attenboroughs david making series behindthescenes natural tv sky went documentary look museum technology
Cluster 10:

In [19]:
# Print the top 10 terms for each cluster
order_centroids_shows = km_shows.cluster_centers_.argsort()[:, ::-1]
terms_shows = tfidf_vectorizer_shows.get_feature_names_out()
for i in range(num_clusters_shows):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids_shows[i, :13]:
        print(' %s' % terms_shows[ind], end='')
    print()

Cluster 0: australia return set art murder crime political join series woman dance story leading
Cluster 1: friend adventure epic little new family world battle join school follow doctor edge
Cluster 2: news abc issue story brings reporting today original exclusive presented matter live investigation
Cluster 3: australian dog australia day incredible life challenge climate journey change like world asking
Cluster 4: life people help series live everyday history stage story way look family new
Cluster 5: world team best war robot explore uk series friend child greatest story game
Cluster 6: series book kid fun stage word adventure based english early love language big
Cluster 7: science australian teach stage lesson technology skill teacher world tenminute experiment fundamental series


In [20]:
joined_activity = pd.merge(df, activities, left_on='id', right_on='content_id', how='right')
#make from list in tag2 column to string and delete [] in the string
joined_activity['tags2'] = joined_activity['tags2'].apply(lambda x: str(x).replace('[', '').replace(']', ''))
#drop NaN values in id column
joined_activity.dropna(subset=['id'], inplace=True)
#drop columns that are not needed
joined_activity.drop(['series', 'episode_name', 'description2', 'publication_date', 'rating', 'season', 'episode', 'episode_title', 'datetime', 'duration_sec', 'tags'], axis=1, inplace=True)
joined_activity.head()

,id,category,title,description,image,directors_actors,tags2,features,cluster,content_id,activity,user_id
3,144.0,Family,Zog,"Based on the picture book by Julia Donaldson, ...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW124...,"[['Director Max Lang', 'Daniel Snaddon']]","'abc4kids', '4feature', '4featured', 'abcme-sh...",based picture book julia donaldson animated co...,5.0,144,Dislike,14
12,246.0,Documentary,Tea with the Dames,"Screen icons Eileen Atkins, Judi Dench, Joan P...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW188...,[['Director Roger Michell']],"'abc1', 'uk', 'docs', 'factual', 'biography', ...",screen icon eileen atkins judi dench joan plow...,9.0,246,Play,10
21,34.0,Movies,Like Minds,"When a schoolboy is suspected of murder, a for...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW290...,[['Director Gregory J. Read']],"'abc1', 'abc2', 'thriller', 'drama', 'crime', ...",schoolboy suspected murder forensic psychologi...,7.0,34,Dislike,15
24,219.0,Documentary,I'm Wanita,"The story of Australia's Queen of Honky Tonk, ...",https://cdn.iview.abc.net.au/thumbs/i/ac/AC182...,[[]],"'abc2', 'abc1', 'docs', 'factual', 'abcarts', ...",story australia queen honky tonk wanita bahtiy...,8.0,219,Like,11
26,328.0,Comedy,The Pirates Of Penzance (1994),This spectacular production of the popular Gil...,https://cdn.iview.abc.net.au/thumbs/i/ae/AE941...,[[]],"'abc2', 'abcarts', 'australia', 'arts', 'class...",spectacular production popular gilbert sulliva...,8.0,328,Dislike,16


In [21]:
joined_activity_shows = pd.merge(df_2, activities, left_on='id', right_on='content_id', how='right')
#make from list in tag2 column to string and delete [] in the string
joined_activity_shows['tags2'] = joined_activity_shows['tags2'].apply(lambda x: str(x).replace('[', '').replace(']', ''))
#drop NaN values in id column
joined_activity_shows.dropna(subset=['id'], inplace=True)
#drop columns that are not needed
joined_activity_shows.drop([ 'description2', 'publication_date', 'rating', 'datetime', 'duration_sec', 'tags'], axis=1, inplace=True)
joined_activity_shows.head()

,level_0,id,title,category,description,directors_actors,tags2,image,features,cluster,content_id,activity,user_id
0,143.0,10143.0,Decoding Danger,Documentary,"Apocalyptic fire storms, raging floods and the...",[[]],"'science', 'informative', 'animals', 'climate'...",https://cdn.iview.abc.net.au/thumbs/i/do/DO195...,apocalyptic fire storm raging flood venomous c...,5.0,10143,Like,14
1,78.0,10078.0,Bed Of Roses,Drama,An Australian comedy drama which centres on Lo...,"['Julia Blake', 'Hanna Mangan Lawrence', ['Cas...","'a:older-sink-dink', 'love', 'comedy', 'abc1',...",https://cdn.iview.abc.net.au/thumbs/i/dr/publi...,australian comedy drama centre louisa atherton...,3.0,10078,Like,11
2,227.0,10227.0,Hitting Home With Sarah Ferguson,Documentary,Award-winning journalist Sarah Ferguson spends...,[['Host Sarah Ferguson']],"'ns:be-informed', 'conversation-starters', 'a:...",https://cdn.iview.abc.net.au/thumbs/i/do/DO140...,awardwinning journalist sarah ferguson spends ...,0.0,10227,Play,14
4,553.0,10553.0,The Wonder Of Marsupials,Family,From Australia's scorching desert heart to pri...,"['Edward Saltau'], ['Director Daniel Hunter']","'ns:be-informed', 'natural-world', 'compelling...",https://cdn.iview.abc.net.au/thumbs/i/ai/AI185...,australia scorching desert heart pristine rain...,0.0,10553,Like,20
5,343.0,10343.0,Muster Dogs,Family,Five graziers from across Australia are given ...,[[]],"'endearing', 'real-life', 'diversity', 'region...",https://cdn.iview.abc.net.au/thumbs/i/rf/RF200...,five grazier across australia given five kelpi...,3.0,10343,Play,16


In [22]:
#select rows where activity is 'Like'
expat_data_likes = joined_activity[joined_activity['activity'] == 'Like']
expat_data_likes.sort_values(by=['user_id'])

,id,category,title,description,image,directors_actors,tags2,features,cluster,content_id,activity,user_id
5835,58.0,Movies,Angel,"Adapted from Elizabeth Taylor's novel, a passi...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW287...,[['Director Francois Ozon']],"'abc1', 'abc2', 'uk', 'europe', 'drama', 'roma...",adapted elizabeth taylor novel passionate youn...,7.0,58,Like,0
4135,30.0,Movies,The Well,Best friends raised on an isolated farm are lo...,https://cdn.iview.abc.net.au/thumbs/i/zy/ZY946...,[['Director Samantha Lang']],"'abc1', 'drama', 'film', '1990s', 'countryside...",best friend raised isolated farm looking futur...,10.0,30,Like,0
5405,20.0,Movies,Five Easy Pieces,After rejecting his wealthy and cultured upbri...,https://cdn.iview.abc.net.au/thumbs/i/zy/ZY928...,[['Director Bob Rafelson']],"'abc1', 'abc2', 'usa', 'americana', '1970s', '...",rejecting wealthy cultured upbringing young ma...,12.0,20,Like,0
5153,337.0,Education,Carbon Cycles And Climate Change In The Tundra,"Filmed in the Canadian Arctic, this program id...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW193...,[[]],"'abc3', 'education', 'secondary-geography'",filmed canadian arctic program identifies expl...,0.0,337,Like,0
6347,319.0,Comedy,Opera On Sydney Harbour: La Boheme,Experience the romance of the original bohemia...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW230...,[['Director Andy Morton']],"'abc2', 'abcarts', 'arts', 'opera', 'performan...",experience romance original bohemian love stor...,8.0,319,Like,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6429,163.0,Family,Legends Of Oz: Dorothy's Return,An animated musical based on the adventure boo...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW041...,"[['Director Daniel St. Pierre', 'Will Finn']]","'abc3', 'childrens', '3featured', 'abc3-faves'...",animated musical based adventure book roger st...,5.0,163,Like,32
6023,155.0,Family,Australia: The Wild Top End,Come with us on an epic journey through some o...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW284...,[[]],"'abc3', '3featured', 'family-viewing-abcme', '...",come u epic journey wildest landscape earth ta...,12.0,155,Like,32
6271,145.0,Family,Armadillo: Narrated By David Attenborough,In the heart of Brazil lives an animal so elus...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW311...,[['Host Sir David Attenborough']],"'abc1', 'uk', 'docs', 'factual', 'nature', 'wi...",heart brazil life animal elusive ever seen wil...,12.0,145,Like,32
6136,34.0,Movies,Like Minds,"When a schoolboy is suspected of murder, a for...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW290...,[['Director Gregory J. Read']],"'abc1', 'abc2', 'thriller', 'drama', 'crime', ...",schoolboy suspected murder forensic psychologi...,7.0,34,Like,32


In [23]:
#select rows where activity is 'Like'
expat_data_shows_likes = joined_activity_shows[joined_activity_shows['activity'] == 'Like']
expat_data_shows_likes.sort_values(by=['user_id'])

,level_0,id,title,category,description,directors_actors,tags2,image,features,cluster,content_id,activity,user_id
7055,103.0,10103.0,Buddi,Kids,Buddi welcomes you into a magical world filled...,[[]],"'abc4kids', '4puggle'",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW240...,buddi welcome magical world filled natural won...,3.0,10103,Like,0
7137,467.0,10467.0,Spicks And Specks,Panel Discussion,"Join Adam Hills, Myf Warhurst and Alan Brough ...","['Myf Warhurst', 'Alan Brough'], ['Hosts Adam ...","'ns:be-entertained', 'a:older-sink-dink', 'wit...",https://cdn.iview.abc.net.au/thumbs/i/le/LE200...,join adam hill myf warhurst alan brough brand ...,4.0,10467,Like,0
4111,512.0,10512.0,The Inbestigators,Family,"When ten-year-old Maudie, a freakishly gifted ...","[['Cast Anna Cooke', 'Jamil Smyth-Secka', 'Ast...","'family-viewing-abcme', 'explore', 'abc3-faves...",https://cdn.iview.abc.net.au/thumbs/i/ck/CK190...,tenyearold maudie freakishly gifted observer h...,0.0,10512,Like,0
6779,580.0,10580.0,Vera,Drama,"DCI Vera Stanhope returns, leading her diligen...","['Kenny Doughty', 'Jon Morrison', 'Ibinabo Jac...","'mystery', 'compelling', 'thriller', 'strong-f...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW314...,dci vera stanhope return leading diligent team...,0.0,10580,Like,0
6985,451.0,10451.0,Sesame Street,Kids,"Join Big Bird, Oscar, Elmo, Bert, Cookie Monst...",[[]],"'happy', '4celebrity', '4best', 'childrens', '...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW162...,join big bird oscar elmo bert cookie monster c...,0.0,10451,Like,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3107,270.0,10270.0,Kids Listen: Story Salad,Kids,Join Matt from Play School and one of his stor...,[[]],'abc4kids',https://cdn.iview.abc.net.au/thumbs/i/ch/CH204...,join matt play school one storyteller friend g...,6.0,10270,Like,30
3217,564.0,10564.0,Tish Tash,Kids,Follow the adventures of a young bear called T...,[[]],"'4girls', 'abc4kids'",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW276...,follow adventure young bear called tish bear f...,1.0,10564,Like,30
3256,507.0,10507.0,The Heights,Drama,Welcome to Arcadia Heights. Whether it's the h...,"['Koa Nuen', ['Cast Marcus Graham', 'Shari Seb...","'relationship-drama', 'love', 'coming-of-age',...",https://cdn.iview.abc.net.au/thumbs/i/dr/DR170...,welcome arcadia height whether highrise high l...,4.0,10507,Like,30
2670,404.0,10404.0,Play School: Through The Windows,Kids,"Through the eyes of a child, explore the world...",[[]],"'christmas', '4pslets-explore', '4aussie', '4u...",https://cdn.iview.abc.net.au/thumbs/i/ck/CK180...,eye child explore world around u full humour w...,5.0,10404,Like,30


In [24]:
#top-3 clusters for each user in expat_data_likes
top3 = expat_data_likes.groupby('user_id')['cluster'].value_counts().groupby(level=0).nlargest(3).reset_index(level=1, drop=True)
#make from series to dataframe
top3 = top3.to_frame()
#drop the 3rd column
top3 = top3.drop(columns=['cluster'])
#make user_id column as index
top3 = top3.reset_index()
top3

,user_id,cluster
0,0,3.0
1,0,7.0
2,0,12.0
3,1,12.0
4,1,3.0
...,...,...
94,31,6.0
95,31,12.0
96,32,12.0
97,32,3.0


In [25]:
expat_data_shows_likes.sort_values(by=['user_id'])
#top-3 clusters for each user in expat_data_likes
top3_show = expat_data_shows_likes.groupby('user_id')['cluster'].value_counts().groupby(level=0).nlargest(3).reset_index(level=1,
                                                                                                              drop=True)
#make from series to dataframe
top3_show = top3_show.to_frame()
#drop the 3rd column
top3_show = top3_show.drop(columns=['cluster'])
#make user_id column as index
top3_show = top3_show.reset_index()
top3_show

,user_id,cluster
0,0,4.0
1,0,0.0
2,0,6.0
3,1,4.0
4,1,0.0
...,...,...
88,29,4.0
89,29,6.0
90,30,4.0
91,30,0.0


In [26]:
#join top 3 clusters for each user with expat_library
recommended= pd.merge(top3, expat_library, on=['cluster'], how='left')
recommended

,user_id,cluster,episode_title,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,features
0,0,3.0,The Mule,24.0,Movies,The Mule,NaN,The Mule,A first-time drug mule tries desperately to pr...,A first-time drug mule tries desperately to pr...,"[ABC TV, MOVIES, DRAMA]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW238...,"[['Director Tony Mahony', 'Angus Sampson']]","[abc1, film, drama, comedy-drama, drugs, organ...",2021-06-11 07:00:00,MA,5883.0,NaN,,firsttime drug mule try desperately prevent bo...
1,0,3.0,Freeman,157.0,Family,Freeman,NaN,Freeman,The story of a nation coming together around I...,The story of a nation coming together around I...,"[ABC TV, ABC ME, INDIGENOUS, DOCUMENTARY, SPOR...",https://cdn.iview.abc.net.au/thumbs/i/rf/RF190...,[['Director Laurence Billiet']],"[abc1, australia, indigenous, docs, factual, f...",2021-06-30 07:05:00,G,3480.0,NaN,,story nation coming together around indigenous...
2,0,3.0,Capturing Cricket: Steve Waugh In India,232.0,Documentary,Capturing Cricket: Steve Waugh In India,NaN,Capturing Cricket: Steve Waugh In India,"Steve Waugh, the mastermind behind Australia's...","Steve Waugh, the mastermind behind Australia's...","[ABC TV, DOCUMENTARY, SPORT]",https://cdn.iview.abc.net.au/thumbs/i/rf/RF192...,[['Host Steve Waugh']],"[abc1, australia, docs, factual, sport, divers...",2020-11-17 21:30:47,PG,3407.0,NaN,,steve waugh mastermind behind australia golden...
3,0,3.0,Brazen Hussies,235.0,Documentary,Brazen Hussies,NaN,Brazen Hussies,A revealing documentary celebrating the legacy...,A revealing documentary celebrating the legacy...,"[ABC TV, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/do/DO190...,[['Director Catherine Dwyer']],"[abc1, featured, docs, australia, factual, his...",2022-03-08 01:04:06,M,5318.0,NaN,,revealing documentary celebrating legacy bold ...
4,0,3.0,Australia Day Live,313.0,Comedy,Australia Day Live 2022,NaN,Australia Day Live 2022,A concert and fireworks spectacular from Sydne...,A concert and fireworks spectacular from Sydne...,"[ABC TV, ABC ARTS, ARTS & CULTURE]",https://cdn.iview.abc.net.au/thumbs/i/rv/RV210...,"[['Host Jeremy Fernandez', 'Casey Donovan', 'J...","[abc1, aussie, australia, event, concert, musi...",2022-01-26 21:31:00,PG,7871.0,NaN,,concert firework spectacular sydney featuring ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,32,5.0,Murray Whelan: Stiff,139.0,Movies,Murray Whelan: Stiff,NaN,Murray Whelan: Stiff,"Murray Whelan, a young political advisers, mus...","Murray Whelan, a young political advisers, mus...","[ABC TV, DRAMA, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW281...,[['Director John Clarke']],"[abc1, australia, drama, crime, crime-drama, 2...",2021-06-25 07:00:00,M,5567.0,NaN,,murray whelan young political adviser must pla...
574,32,5.0,Stargazing: Moon and Beyond,167.0,Family,Stargazing: Moon and Beyond,NaN,Stargazing: Moon and Beyond,Stargazing celebrates the 50th anniversary of ...,Stargazing celebrates the 50th anniversary of ...,"[ABC TV, SCIENCE, DOCUMENTARY, FAMILY, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/sc/SC180...,"[['Host Julia Zemiro', 'Brian Cox']]","[abc1, science, docs, space, family-viewing, e...",2019-07-16 21:01:50,PG,3599.0,NaN,,stargazing celebrates 50th anniversary moon la...
575,32,5.0,Cyber Dreaming,218.0,Documentary,Cyber Dreaming,NaN,Cyber Dreaming,Graphic artist Stuart Campbell has developed h...,Graphic artist Stuart Campbell has developed h...,"[ABC TV, ABC ARTS, ARTS & CULTURE, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/ac/publi...,[[]],"[abc1, abcarts, arts, docs, aussie, naidoc, hi...",2018-07-11 14:00:50,G,1570.0,NaN,,graphic artist stuart campbell developed uniqu...
576,32,5.0,Step Into Paradise,256.0,Documentary,Step Into Paradise,NaN,Step Into Paradise,The extraordinary story of iconic fashion desi...,The extraordinary story of iconic fashi

In [27]:
#shows
recommended_shows= pd.merge(top3_show, expat_library_shows, on=['cluster'], how='left')
recommended_shows

,user_id,cluster,index,id,title,category,description,description2,directors_actors,publication_date,rating,duration_sec,tags,tags2,image,features
0,0,4.0,58.0,10058.0,Australia's Ocean Odyssey: A Journey Down The ...,Family,A landmark documentary series that takes a spe...,[Journey down the East Australian Current all ...,[['Narrator Marta Dusseldorp']],2020-06-09 21:33:25,G,"[3549.0, 360.0, 3515.0, 3446.0]","[ABC TV, ABC ME, DOCUMENTARY, SCIENCE, FAMILY]","[ocean, informative, natural-world, animals, s...",https://cdn.iview.abc.net.au/thumbs/i/do/DO180...,landmark documentary series take spectacular j...
1,0,4.0,61.0,10061.0,Australian Story,News,"Putting the 'real' back into reality TV, the a...",[A daring sea rescue led by the Australian Nav...,[[]],2021-08-23 20:31:30,None,"[1856.0, 1920.0, 1837.0, 1782.0, 2070.0, 1693....","[ABC TV, ABC NEWS, DOCUMENTARY]","[natural-world, politics, bob-hawke, sport, or...",https://cdn.iview.abc.net.au/thumbs/i/nc/NC210...,putting real back reality tv awardwinning seri...
2,0,4.0,144.0,10144.0,Deep Dive Into Australia's Ocean Odyssey,Education,"Focussing on marine life, interconnectedness o...",[Emily Jateff from Australian National Maritim...,[[]],2020-08-17 11:56:21,PG,"[1206.0, 1090.0, 1271.0]","[ABC ME, EDUCATION]","[secondary-maths, education, abc3]",https://cdn.iview.abc.net.au/thumbs/i/ck/CK203...,focussing marine life interconnectedness land ...
3,0,0.0,54.0,10054.0,Australia Remastered: Nature's Great Divide,Family,The hard line that separates the natural world...,[The narrow strait between the Indonesian isla...,[['Host Aaron Pedersen']],2021-12-07 21:27:05,G,"[3240.0, 3240.0, 3200.0]","[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]","[ocean, informative, natural-world, animals, f...",https://cdn.iview.abc.net.au/thumbs/i/do/DO195...,hard line separate natural world asia australi...
4,0,0.0,56.0,10056.0,Australia Remastered: Wild Treasures,Family,Australia's diverse and unique landscapes and ...,[The Daintree Rainforest has one of the highes...,[['Host Aaron Pedersen']],2021-01-11 21:22:47,G,"[3205.0, 3315.0, 3260.0, 3254.0, 3257.0, 3220.0]","[ABC TV, ABC ME, DOCUMENTARY, FAMILY]","[natural-world, wildlife, family-viewing-abcme...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,australia diverse unique landscape ecosystem u...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,30,0.0,485.0,10485.0,The Art Of Australia,Documentary,The Art of Australia explores how art and arti...,[The story of how art helped European settlers...,[['Host Edmund Capon']],2021-06-18 07:00:00,G,"[3390.0, 3423.0, 3402.0]","[ABC TV, ABC ARTS, ARTS & CULTURE, DOCUMENTARY]","[informative, a:older-sink-dink, ns:be-inspire...",https://cdn.iview.abc.net.au/thumbs/i/ac/AC123...,art australia explores art artist helped shape...
330,30,0.0,495.0,10495.0,The Crown and Us: The Story of The Royals in A...,Documentary,The Crown is part of who we were and who we ar...,[Explore the relationship between Australia an...,[['Director Bernice Toni']],2021-06-18 07:00:00,PG,"[3463.0, 3434.0]","[ABC TV, ABC TV Plus, DOCUMENTARY]","[real-life, informative, nostalgia, factual, c...",https://cdn.iview.abc.net.au/thumbs/i/do/DO171...,crown part despite secret scandal question rel...
331,30,5.0,195.0,10195.0,Gardening Australia - My Garden Path,Comedy,Gardening Australia is hosted by Costa Georgia...,[We meet a horticulturalist and indoor plant c...,[[]],2021-02-12 07:00:00,G,"[372.0, 366.0, 509.0, 407.0, 385.0, 467.0, 401...","[ABC TV, LIFESTYLE, REGIONAL AUSTRALIA]","[diversity, melbourne, australian-capital-terr...",https://cdn.iview.abc.net.au/thumbs/i/rk/RK190...,gardening australia hosted costa georgiadis sc...
332,30,5.0,408.0,10408.0,Prepping Australia,Documentary,Meet real Australian Preppers bunkering down f...,[After the 2020 supermarket shortages left her...,[[]],2021-11-16 22:30:00,G,"[601.0, 909.0, 934.0]","[ABC TV Plus, DOCUMENTARY]","[real-life, factual, docuseries, quirky, unbel...",https://cdn.

In [28]:
recommended_div= pd.merge(top3, diversity_library, on=['cluster'], how='left')
recommended_shows_div= pd.merge(top3_show, diversity_library_shows, on=['cluster'], how='left')
recommended_div

,user_id,cluster,episode_title,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,features
0,0,3.0,Capturing Cricket: Steve Waugh In India,232.0,Documentary,Capturing Cricket: Steve Waugh In India,NaN,Capturing Cricket: Steve Waugh In India,"Steve Waugh, the mastermind behind Australia's...","Steve Waugh, the mastermind behind Australia's...","[ABC TV, DOCUMENTARY, SPORT]",https://cdn.iview.abc.net.au/thumbs/i/rf/RF192...,[['Host Steve Waugh']],"[abc1, australia, docs, factual, sport, divers...",2020-11-17 21:30:47,PG,3407.0,NaN,,steve waugh mastermind behind australia golden...
1,0,3.0,Australia Day Live,313.0,Comedy,Australia Day Live 2022,NaN,Australia Day Live 2022,A concert and fireworks spectacular from Sydne...,A concert and fireworks spectacular from Sydne...,"[ABC TV, ABC ARTS, ARTS & CULTURE]",https://cdn.iview.abc.net.au/thumbs/i/rv/RV210...,"[['Host Jeremy Fernandez', 'Casey Donovan', 'J...","[abc1, aussie, australia, event, concert, musi...",2022-01-26 21:31:00,PG,7871.0,NaN,,concert firework spectacular sydney featuring ...
2,0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,31,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
160,32,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
161,32,3.0,Capturing Cricket: Steve Waugh In India,232.0,Documentary,Capturing Cricket: Steve Waugh In India,NaN,Capturing Cricket: Steve Waugh In India,"Steve Waugh, the mastermind behind Australia's...","Steve Waugh, the mastermind behind Australia's...","[ABC TV, DOCUMENTARY, SPORT]",https://cdn.iview.abc.net.au/thumbs/i/rf/RF192...,[['Host Steve Waugh']],"[abc1, australia, docs, factual, sport, divers...",2020-11-17 21:30:47,PG,3407.0,NaN,,steve waugh mastermind behind australia golden...
162,32,3.0,Australia Day Live,313.0,Comedy,Australia Day Live 2022,NaN,Australia Day Live 2022,A concert and fireworks spectacular from Sydne...,A concert and fireworks spectacular from Sydne...,"[ABC TV, ABC ARTS, ARTS & CULTURE]",https://cdn.iview.abc.net.au/thumbs/i/rv/RV210...,"[['Host Jeremy Fernandez', 'Casey Donovan', 'J...","[abc1, aussie, australia, event, concert, musi...",2022-01-26 21:31:00,PG,7871.0,NaN,,concert firework spectacular sydney featuring ...


In [29]:
#drop intersections of recommended and activity by user_id and content_id
recommended = recommended[~recommended[['user_id', 'id']].apply(frozenset, axis=1).isin(activities[['user_id', 'content_id']].apply(frozenset, axis=1))]
recommended

,user_id,cluster,episode_title,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,features
0,0,3.0,The Mule,24.0,Movies,The Mule,NaN,The Mule,A first-time drug mule tries desperately to pr...,A first-time drug mule tries desperately to pr...,"[ABC TV, MOVIES, DRAMA]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW238...,"[['Director Tony Mahony', 'Angus Sampson']]","[abc1, film, drama, comedy-drama, drugs, organ...",2021-06-11 07:00:00,MA,5883.0,NaN,,firsttime drug mule try desperately prevent bo...
1,0,3.0,Freeman,157.0,Family,Freeman,NaN,Freeman,The story of a nation coming together around I...,The story of a nation coming together around I...,"[ABC TV, ABC ME, INDIGENOUS, DOCUMENTARY, SPOR...",https://cdn.iview.abc.net.au/thumbs/i/rf/RF190...,[['Director Laurence Billiet']],"[abc1, australia, indigenous, docs, factual, f...",2021-06-30 07:05:00,G,3480.0,NaN,,story nation coming together around indigenous...
2,0,3.0,Capturing Cricket: Steve Waugh In India,232.0,Documentary,Capturing Cricket: Steve Waugh In India,NaN,Capturing Cricket: Steve Waugh In India,"Steve Waugh, the mastermind behind Australia's...","Steve Waugh, the mastermind behind Australia's...","[ABC TV, DOCUMENTARY, SPORT]",https://cdn.iview.abc.net.au/thumbs/i/rf/RF192...,[['Host Steve Waugh']],"[abc1, australia, docs, factual, sport, divers...",2020-11-17 21:30:47,PG,3407.0,NaN,,steve waugh mastermind behind australia golden...
3,0,3.0,Brazen Hussies,235.0,Documentary,Brazen Hussies,NaN,Brazen Hussies,A revealing documentary celebrating the legacy...,A revealing documentary celebrating the legacy...,"[ABC TV, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/do/DO190...,[['Director Catherine Dwyer']],"[abc1, featured, docs, australia, factual, his...",2022-03-08 01:04:06,M,5318.0,NaN,,revealing documentary celebrating legacy bold ...
5,0,3.0,Opera on Sydney Harbour: Carmen,322.0,Comedy,Opera on Sydney Harbour: Carmen,NaN,Opera on Sydney Harbour: Carmen,"Opera Australia performs Bizet's Carmen, again...","Opera Australia performs Bizet's Carmen, again...","[ABC TV, ABC ARTS, ARTS & CULTURE]",https://cdn.iview.abc.net.au/thumbs/i/zx/ZX954...,[['Director Cameron Kirkpatrick']],"[abc1, abcarts, arts, opera, performance, love...",2021-06-18 07:00:00,PG,8467.0,NaN,,opera australia performs bizet carmen backdrop...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,32,5.0,Murray Whelan: Stiff,139.0,Movies,Murray Whelan: Stiff,NaN,Murray Whelan: Stiff,"Murray Whelan, a young political advisers, mus...","Murray Whelan, a young political advisers, mus...","[ABC TV, DRAMA, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW281...,[['Director John Clarke']],"[abc1, australia, drama, crime, crime-drama, 2...",2021-06-25 07:00:00,M,5567.0,NaN,,murray whelan young political adviser must pla...
574,32,5.0,Stargazing: Moon and Beyond,167.0,Family,Stargazing: Moon and Beyond,NaN,Stargazing: Moon and Beyond,Stargazing celebrates the 50th anniversary of ...,Stargazing celebrates the 50th anniversary of ...,"[ABC TV, SCIENCE, DOCUMENTARY, FAMILY, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/sc/SC180...,"[['Host Julia Zemiro', 'Brian Cox']]","[abc1, science, docs, space, family-viewing, e...",2019-07-16 21:01:50,PG,3599.0,NaN,,stargazing celebrates 50th anniversary moon la...
575,32,5.0,Cyber Dreaming,218.0,Documentary,Cyber Dreaming,NaN,Cyber Dreaming,Graphic artist Stuart Campbell has developed h...,Graphic artist Stuart Campbell has developed h...,"[ABC TV, ABC ARTS, ARTS & CULTURE, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/ac/publi...,[[]],"[abc1, abcarts, arts, docs, aussie, naidoc, hi...",2018-07-11 14:00:50,G,1570.0,NaN,,graphic artist stuart campbell developed uniqu...
576,32,5.0,Step Into Paradise,256.0,Documentary,Step Into Paradise,NaN,Step Into Paradise,The extraordinary story of iconic fashion desi...,The extraordinary story

In [30]:
recommended_shows = recommended_shows[~recommended_shows[['user_id', 'id']].apply(frozenset, axis=1).isin(activities[['user_id', 'content_id']].apply(frozenset, axis=1))]
recommended_shows

,user_id,cluster,index,id,title,category,description,description2,directors_actors,publication_date,rating,duration_sec,tags,tags2,image,features
0,0,4.0,58.0,10058.0,Australia's Ocean Odyssey: A Journey Down The ...,Family,A landmark documentary series that takes a spe...,[Journey down the East Australian Current all ...,[['Narrator Marta Dusseldorp']],2020-06-09 21:33:25,G,"[3549.0, 360.0, 3515.0, 3446.0]","[ABC TV, ABC ME, DOCUMENTARY, SCIENCE, FAMILY]","[ocean, informative, natural-world, animals, s...",https://cdn.iview.abc.net.au/thumbs/i/do/DO180...,landmark documentary series take spectacular j...
2,0,4.0,144.0,10144.0,Deep Dive Into Australia's Ocean Odyssey,Education,"Focussing on marine life, interconnectedness o...",[Emily Jateff from Australian National Maritim...,[[]],2020-08-17 11:56:21,PG,"[1206.0, 1090.0, 1271.0]","[ABC ME, EDUCATION]","[secondary-maths, education, abc3]",https://cdn.iview.abc.net.au/thumbs/i/ck/CK203...,focussing marine life interconnectedness land ...
4,0,0.0,56.0,10056.0,Australia Remastered: Wild Treasures,Family,Australia's diverse and unique landscapes and ...,[The Daintree Rainforest has one of the highes...,[['Host Aaron Pedersen']],2021-01-11 21:22:47,G,"[3205.0, 3315.0, 3260.0, 3254.0, 3257.0, 3220.0]","[ABC TV, ABC ME, DOCUMENTARY, FAMILY]","[natural-world, wildlife, family-viewing-abcme...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,australia diverse unique landscape ecosystem u...
7,0,0.0,495.0,10495.0,The Crown and Us: The Story of The Royals in A...,Documentary,The Crown is part of who we were and who we ar...,[Explore the relationship between Australia an...,[['Director Bernice Toni']],2021-06-18 07:00:00,PG,"[3463.0, 3434.0]","[ABC TV, ABC TV Plus, DOCUMENTARY]","[real-life, informative, nostalgia, factual, c...",https://cdn.iview.abc.net.au/thumbs/i/do/DO171...,crown part despite secret scandal question rel...
11,1,4.0,61.0,10061.0,Australian Story,News,"Putting the 'real' back into reality TV, the a...",[A daring sea rescue led by the Australian Nav...,[[]],2021-08-23 20:31:30,None,"[1856.0, 1920.0, 1837.0, 1782.0, 2070.0, 1693....","[ABC TV, ABC NEWS, DOCUMENTARY]","[natural-world, politics, bob-hawke, sport, or...",https://cdn.iview.abc.net.au/thumbs/i/nc/NC210...,putting real back reality tv awardwinning seri...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,30,0.0,485.0,10485.0,The Art Of Australia,Documentary,The Art of Australia explores how art and arti...,[The story of how art helped European settlers...,[['Host Edmund Capon']],2021-06-18 07:00:00,G,"[3390.0, 3423.0, 3402.0]","[ABC TV, ABC ARTS, ARTS & CULTURE, DOCUMENTARY]","[informative, a:older-sink-dink, ns:be-inspire...",https://cdn.iview.abc.net.au/thumbs/i/ac/AC123...,art australia explores art artist helped shape...
330,30,0.0,495.0,10495.0,The Crown and Us: The Story of The Royals in A...,Documentary,The Crown is part of who we were and who we ar...,[Explore the relationship between Australia an...,[['Director Bernice Toni']],2021-06-18 07:00:00,PG,"[3463.0, 3434.0]","[ABC TV, ABC TV Plus, DOCUMENTARY]","[real-life, informative, nostalgia, factual, c...",https://cdn.iview.abc.net.au/thumbs/i/do/DO171...,crown part despite secret scandal question rel...
331,30,5.0,195.0,10195.0,Gardening Australia - My Garden Path,Comedy,Gardening Australia is hosted by Costa Georgia...,[We meet a horticulturalist and indoor plant c...,[[]],2021-02-12 07:00:00,G,"[372.0, 366.0, 509.0, 407.0, 385.0, 467.0, 401...","[ABC TV, LIFESTYLE, REGIONAL AUSTRALIA]","[diversity, melbourne, australian-capital-terr...",https://cdn.iview.abc.net.au/thumbs/i/rk/RK190...,gardening australia hosted costa georgiadis sc...
332,30,5.0,408.0,10408.0,Prepping Australia,Documentary,Meet real Australian Preppers bunkering down f...,[After the 2020 supermarket shortages left her...,[[]],2021-11-16 22:30:00,G,"[601.0, 909.0, 934.0]","[ABC TV Plus, DOCUMENTARY]","[real-life, factual, docuseries, quirky, unbel...",https://

In [31]:
recommended_div = recommended_div[~recommended_div[['user_id', 'id']].apply(frozenset, axis=1).isin(activities[['user_id', 'content_id']].apply(frozenset, axis=1))]
recommended_div

,user_id,cluster,episode_title,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,features
0,0,3.0,Capturing Cricket: Steve Waugh In India,232.0,Documentary,Capturing Cricket: Steve Waugh In India,NaN,Capturing Cricket: Steve Waugh In India,"Steve Waugh, the mastermind behind Australia's...","Steve Waugh, the mastermind behind Australia's...","[ABC TV, DOCUMENTARY, SPORT]",https://cdn.iview.abc.net.au/thumbs/i/rf/RF192...,[['Host Steve Waugh']],"[abc1, australia, docs, factual, sport, divers...",2020-11-17 21:30:47,PG,3407.0,NaN,,steve waugh mastermind behind australia golden...
2,0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,3.0,Capturing Cricket: Steve Waugh In India,232.0,Documentary,Capturing Cricket: Steve Waugh In India,NaN,Capturing Cricket: Steve Waugh In India,"Steve Waugh, the mastermind behind Australia's...","Steve Waugh, the mastermind behind Australia's...","[ABC TV, DOCUMENTARY, SPORT]",https://cdn.iview.abc.net.au/thumbs/i/rf/RF192...,[['Host Steve Waugh']],"[abc1, australia, docs, factual, sport, divers...",2020-11-17 21:30:47,PG,3407.0,NaN,,steve waugh mastermind behind australia golden...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,31,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
160,32,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
161,32,3.0,Capturing Cricket: Steve Waugh In India,232.0,Documentary,Capturing Cricket: Steve Waugh In India,NaN,Capturing Cricket: Steve Waugh In India,"Steve Waugh, the mastermind behind Australia's...","Steve Waugh, the mastermind behind Australia's...","[ABC TV, DOCUMENTARY, SPORT]",https://cdn.iview.abc.net.au/thumbs/i/rf/RF192...,[['Host Steve Waugh']],"[abc1, australia, docs, factual, sport, divers...",2020-11-17 21:30:47,PG,3407.0,NaN,,steve waugh mastermind behind australia golden...
162,32,3.0,Australia Day Live,313.0,Comedy,Australia Day Live 2022,NaN,Australia Day Live 2022,A concert and fireworks spectacular from Sydne...,A concert and fireworks spectacular from Sydne...,"[ABC TV, ABC ARTS, ARTS & CULTURE]",https://cdn.iview.abc.net.au/thumbs/i/rv/RV210...,"[['Host Jeremy Fernandez', 'Casey Donovan', 'J...","[abc1, aussie, australia, event, concert, musi...",2022-01-26 21:31:00,PG,7871.0,NaN,,concert firework spectacular sydney featuring ...


In [32]:
recommended_shows_div = recommended_shows_div[~recommended_shows_div[['user_id', 'id']].apply(frozenset, axis=1).isin(activities[['user_id', 'content_id']].apply(frozenset, axis=1))]
recommended_shows_div

,user_id,cluster,index,id,title,category,description,description2,directors_actors,publication_date,rating,duration_sec,tags,tags2,image,features
1,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,4.0,572.0,10572.0,Traces,Drama,"While attending an online forensic course, a y...",[While Kathy oversees the excavation of Marie'...,"[['Cast Molly Windsor', 'Jennifer Spence', 'La...",2021-08-15 21:27:00,M,"[2618.0, 2592.0, 2712.0, 2642.0, 2581.0, 2593.0]","[ABC TV, DRAMA]","[crime-drama, gritty, mystery, suspense, thril...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW296...,attending online forensic course young lab ass...
4,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,29,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,29,4.0,572.0,10572.0,Traces,Drama,"While attending an online forensic course, a y...",[While Kathy oversees the excavation of Marie'...,"[['Cast Molly Windsor', 'Jennifer Spence', 'La...",2021-08-15 21:27:00,M,"[2618.0, 2592.0, 2712.0, 2642.0, 2581.0, 2593.0]","[ABC TV, DRAMA]","[crime-drama, gritty, mystery, suspense, thril...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW296...,attending online forensic course young lab ass...
89,29,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,30,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
#choose 2 rows per each user_id and each cluster
recommended_films = recommended.groupby(['user_id', 'cluster']).head(2)
recommended_films

,user_id,cluster,episode_title,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,features
0,0,3.0,The Mule,24.0,Movies,The Mule,NaN,The Mule,A first-time drug mule tries desperately to pr...,A first-time drug mule tries desperately to pr...,"[ABC TV, MOVIES, DRAMA]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW238...,"[['Director Tony Mahony', 'Angus Sampson']]","[abc1, film, drama, comedy-drama, drugs, organ...",2021-06-11 07:00:00,MA,5883.0,NaN,,firsttime drug mule try desperately prevent bo...
1,0,3.0,Freeman,157.0,Family,Freeman,NaN,Freeman,The story of a nation coming together around I...,The story of a nation coming together around I...,"[ABC TV, ABC ME, INDIGENOUS, DOCUMENTARY, SPOR...",https://cdn.iview.abc.net.au/thumbs/i/rf/RF190...,[['Director Laurence Billiet']],"[abc1, australia, indigenous, docs, factual, f...",2021-06-30 07:05:00,G,3480.0,NaN,,story nation coming together around indigenous...
6,0,7.0,Stackorama!,153.0,Family,Stackorama!,NaN,Stackorama!,"Jaydyn Coggins, the best cup stacker in Austra...","Jaydyn Coggins, the best cup stacker in Austra...","[ABC TV Plus, DOCUMENTARY, SPORT, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/do/DO191...,[[]],"[abc2, docs, factual, sport, endearing, disabi...",2021-03-31 21:45:07,PG,3586.0,NaN,,jaydyn coggins best cup stacker australia trav...
7,0,7.0,Sea Lions: Life By A Whisker,187.0,Family,Sea Lions: Life By A Whisker,NaN,Sea Lions: Life By A Whisker,Take an intimate journey inside the colony of ...,Take an intimate journey inside the colony of ...,"[ABC ME, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW284...,[['Narrator Sam Neill']],"[abc3, family-viewing-abcme, 3featured, family...",2022-03-16 09:45:00,G,2439.0,NaN,,take intimate journey inside colony australian...
11,0,12.0,Designing A Legacy,223.0,Documentary,Designing A Legacy,NaN,Designing A Legacy,"Comedian, architecture enthusiast and design n...","Comedian, architecture enthusiast and design n...","[ABC TV, LIFESTYLE, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/fa/FA190...,[['Host Tim Ross']],"[abc1, australia, lifestyle, home, creative, a...",2021-02-02 21:03:51,M,3422.0,NaN,,comedian architecture enthusiast design nerd t...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563,32,12.0,Weird Australia,213.0,Documentary,Weird Australia,NaN,Weird Australia,"With the help of eyewitness accounts, qualifie...","With the help of eyewitness accounts, qualifie...","[ABC TV, SCIENCE, REGIONAL AUSTRALIA, DOCUMENT...",https://cdn.iview.abc.net.au/thumbs/i/sq/SQ210...,[['Host Moataz Hamde']],"[abc1, science, australia, nature, wildlife, e...",2022-01-01 09:29:00,PG,1802.0,NaN,,help eyewitness account qualified expert amate...
566,32,3.0,The Mule,24.0,Movies,The Mule,NaN,The Mule,A first-time drug mule tries desperately to pr...,A first-time drug mule tries desperately to pr...,"[ABC TV, MOVIES, DRAMA]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW238...,"[['Director Tony Mahony', 'Angus Sampson']]","[abc1, film, drama, comedy-drama, drugs, organ...",2021-06-11 07:00:00,MA,5883.0,NaN,,firsttime drug mule try desperately prevent bo...
567,32,3.0,Freeman,157.0,Family,Freeman,NaN,Freeman,The story of a nation coming together around I...,The story of a nation coming together around I...,"[ABC TV, ABC ME, INDIGENOUS, DOCUMENTARY, SPOR...",https://cdn.iview.abc.net.au/thumbs/i/rf/RF190...,[['Director Laurence Billiet']],"[abc1, australia, indigenous, docs, factual, f...",2021-06-30 07:05:00,G,3480.0,NaN,,story nation coming together around indigenous...
573,32,5.0,Murray Whelan: Stiff,139.0,Movies,Murray Whelan: Stiff,NaN,Murray Whelan: Stiff,"Murray Whelan, a young political advisers, mus...","Murray Whelan, a young political advisers, mus...","[ABC TV, DRAMA, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW281...,[['Director John Clarke']],"[abc1, australia, drama, crime, crime-

In [34]:
recommended_shows = recommended_shows.groupby(['user_id', 'cluster']).head(2)
recommended_shows

,user_id,cluster,index,id,title,category,description,description2,directors_actors,publication_date,rating,duration_sec,tags,tags2,image,features
0,0,4.0,58.0,10058.0,Australia's Ocean Odyssey: A Journey Down The ...,Family,A landmark documentary series that takes a spe...,[Journey down the East Australian Current all ...,[['Narrator Marta Dusseldorp']],2020-06-09 21:33:25,G,"[3549.0, 360.0, 3515.0, 3446.0]","[ABC TV, ABC ME, DOCUMENTARY, SCIENCE, FAMILY]","[ocean, informative, natural-world, animals, s...",https://cdn.iview.abc.net.au/thumbs/i/do/DO180...,landmark documentary series take spectacular j...
2,0,4.0,144.0,10144.0,Deep Dive Into Australia's Ocean Odyssey,Education,"Focussing on marine life, interconnectedness o...",[Emily Jateff from Australian National Maritim...,[[]],2020-08-17 11:56:21,PG,"[1206.0, 1090.0, 1271.0]","[ABC ME, EDUCATION]","[secondary-maths, education, abc3]",https://cdn.iview.abc.net.au/thumbs/i/ck/CK203...,focussing marine life interconnectedness land ...
4,0,0.0,56.0,10056.0,Australia Remastered: Wild Treasures,Family,Australia's diverse and unique landscapes and ...,[The Daintree Rainforest has one of the highes...,[['Host Aaron Pedersen']],2021-01-11 21:22:47,G,"[3205.0, 3315.0, 3260.0, 3254.0, 3257.0, 3220.0]","[ABC TV, ABC ME, DOCUMENTARY, FAMILY]","[natural-world, wildlife, family-viewing-abcme...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,australia diverse unique landscape ecosystem u...
7,0,0.0,495.0,10495.0,The Crown and Us: The Story of The Royals in A...,Documentary,The Crown is part of who we were and who we ar...,[Explore the relationship between Australia an...,[['Director Bernice Toni']],2021-06-18 07:00:00,PG,"[3463.0, 3434.0]","[ABC TV, ABC TV Plus, DOCUMENTARY]","[real-life, informative, nostalgia, factual, c...",https://cdn.iview.abc.net.au/thumbs/i/do/DO171...,crown part despite secret scandal question rel...
11,1,4.0,61.0,10061.0,Australian Story,News,"Putting the 'real' back into reality TV, the a...",[A daring sea rescue led by the Australian Nav...,[[]],2021-08-23 20:31:30,None,"[1856.0, 1920.0, 1837.0, 1782.0, 2070.0, 1693....","[ABC TV, ABC NEWS, DOCUMENTARY]","[natural-world, politics, bob-hawke, sport, or...",https://cdn.iview.abc.net.au/thumbs/i/nc/NC210...,putting real back reality tv awardwinning seri...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,30,4.0,61.0,10061.0,Australian Story,News,"Putting the 'real' back into reality TV, the a...",[A daring sea rescue led by the Australian Nav...,[[]],2021-08-23 20:31:30,None,"[1856.0, 1920.0, 1837.0, 1782.0, 2070.0, 1693....","[ABC TV, ABC NEWS, DOCUMENTARY]","[natural-world, politics, bob-hawke, sport, or...",https://cdn.iview.abc.net.au/thumbs/i/nc/NC210...,putting real back reality tv awardwinning seri...
326,30,0.0,54.0,10054.0,Australia Remastered: Nature's Great Divide,Family,The hard line that separates the natural world...,[The narrow strait between the Indonesian isla...,[['Host Aaron Pedersen']],2021-12-07 21:27:05,G,"[3240.0, 3240.0, 3200.0]","[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]","[ocean, informative, natural-world, animals, f...",https://cdn.iview.abc.net.au/thumbs/i/do/DO195...,hard line separate natural world asia australi...
327,30,0.0,56.0,10056.0,Australia Remastered: Wild Treasures,Family,Australia's diverse and unique landscapes and ...,[The Daintree Rainforest has one of the highes...,[['Host Aaron Pedersen']],2021-01-11 21:22:47,G,"[3205.0, 3315.0, 3260.0, 3254.0, 3257.0, 3220.0]","[ABC TV, ABC ME, DOCUMENTARY, FAMILY]","[natural-world, wildlife, family-viewing-abcme...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,australia diverse unique landscape ecosystem u...
331,30,5.0,195.0,10195.0,Gardening Australia - My Garden Path,Comedy,Gardening Australia is hosted by Costa Georgia...,[We meet a horticulturalist and indoor plant c...,[[]],2021-02-12 07:00:00,G,"[372.0, 366.0, 509.0, 407.0, 385.0, 467.0, 401...","[ABC TV, LIFESTYLE, REGIONAL AUSTRALIA]","[diversi

In [35]:
recommended_films_div = recommended_div.groupby(['user_id', 'cluster']).head(2)
recommended_films_div

,user_id,cluster,episode_title,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,features
0,0,3.0,Capturing Cricket: Steve Waugh In India,232.0,Documentary,Capturing Cricket: Steve Waugh In India,NaN,Capturing Cricket: Steve Waugh In India,"Steve Waugh, the mastermind behind Australia's...","Steve Waugh, the mastermind behind Australia's...","[ABC TV, DOCUMENTARY, SPORT]",https://cdn.iview.abc.net.au/thumbs/i/rf/RF192...,[['Host Steve Waugh']],"[abc1, australia, docs, factual, sport, divers...",2020-11-17 21:30:47,PG,3407.0,NaN,,steve waugh mastermind behind australia golden...
2,0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,3.0,Capturing Cricket: Steve Waugh In India,232.0,Documentary,Capturing Cricket: Steve Waugh In India,NaN,Capturing Cricket: Steve Waugh In India,"Steve Waugh, the mastermind behind Australia's...","Steve Waugh, the mastermind behind Australia's...","[ABC TV, DOCUMENTARY, SPORT]",https://cdn.iview.abc.net.au/thumbs/i/rf/RF192...,[['Host Steve Waugh']],"[abc1, australia, docs, factual, sport, divers...",2020-11-17 21:30:47,PG,3407.0,NaN,,steve waugh mastermind behind australia golden...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,31,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
160,32,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
161,32,3.0,Capturing Cricket: Steve Waugh In India,232.0,Documentary,Capturing Cricket: Steve Waugh In India,NaN,Capturing Cricket: Steve Waugh In India,"Steve Waugh, the mastermind behind Australia's...","Steve Waugh, the mastermind behind Australia's...","[ABC TV, DOCUMENTARY, SPORT]",https://cdn.iview.abc.net.au/thumbs/i/rf/RF192...,[['Host Steve Waugh']],"[abc1, australia, docs, factual, sport, divers...",2020-11-17 21:30:47,PG,3407.0,NaN,,steve waugh mastermind behind australia golden...
162,32,3.0,Australia Day Live,313.0,Comedy,Australia Day Live 2022,NaN,Australia Day Live 2022,A concert and fireworks spectacular from Sydne...,A concert and fireworks spectacular from Sydne...,"[ABC TV, ABC ARTS, ARTS & CULTURE]",https://cdn.iview.abc.net.au/thumbs/i/rv/RV210...,"[['Host Jeremy Fernandez', 'Casey Donovan', 'J...","[abc1, aussie, australia, event, concert, musi...",2022-01-26 21:31:00,PG,7871.0,NaN,,concert firework spectacular sydney featuring ...


In [36]:
recommended_shows_div = recommended_shows_div.groupby(['user_id', 'cluster']).head(2)
recommended_shows_div

,user_id,cluster,index,id,title,category,description,description2,directors_actors,publication_date,rating,duration_sec,tags,tags2,image,features
1,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,4.0,572.0,10572.0,Traces,Drama,"While attending an online forensic course, a y...",[While Kathy oversees the excavation of Marie'...,"[['Cast Molly Windsor', 'Jennifer Spence', 'La...",2021-08-15 21:27:00,M,"[2618.0, 2592.0, 2712.0, 2642.0, 2581.0, 2593.0]","[ABC TV, DRAMA]","[crime-drama, gritty, mystery, suspense, thril...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW296...,attending online forensic course young lab ass...
4,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,29,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,29,4.0,572.0,10572.0,Traces,Drama,"While attending an online forensic course, a y...",[While Kathy oversees the excavation of Marie'...,"[['Cast Molly Windsor', 'Jennifer Spence', 'La...",2021-08-15 21:27:00,M,"[2618.0, 2592.0, 2712.0, 2642.0, 2581.0, 2593.0]","[ABC TV, DRAMA]","[crime-drama, gritty, mystery, suspense, thril...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW296...,attending online forensic course young lab ass...
89,29,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,30,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
#drop na in id column and mix recommended_films and recommended_shows
# recommended_films.dropna(subset=['id'], inplace=True)
# recommended_shows.dropna(subset=['id'], inplace=True)
#join recommended_films and recommended_shows
recommended_mix_austalia = pd.concat([recommended_films, recommended_shows])
#choose the random 6 rows per each user_id
recommended_mix_austalia = recommended_mix_austalia.groupby('user_id').apply(lambda x: x.sample(2, replace=False))
recommended_mix_austalia.reset_index(drop=True, inplace=True)


In [38]:
recommended_mix_austalia.dropna(subset=['id'], inplace=True)
recommended_mix_austalia

,user_id,cluster,episode_title,id,category,title,series,episode_name,description,description2,...,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,features,index
0,0,0.0,NaN,10056.0,Family,Australia Remastered: Wild Treasures,NaN,NaN,Australia's diverse and unique landscapes and ...,[The Daintree Rainforest has one of the highes...,...,https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,[['Host Aaron Pedersen']],"[natural-world, wildlife, family-viewing-abcme...",2021-01-11 21:22:47,G,"[3205.0, 3315.0, 3260.0, 3254.0, 3257.0, 3220.0]",NaN,NaN,australia diverse unique landscape ecosystem u...,56.0
1,0,12.0,Designing A Legacy,223.0,Documentary,Designing A Legacy,NaN,Designing A Legacy,"Comedian, architecture enthusiast and design n...","Comedian, architecture enthusiast and design n...",...,https://cdn.iview.abc.net.au/thumbs/i/fa/FA190...,[['Host Tim Ross']],"[abc1, australia, lifestyle, home, creative, a...",2021-02-02 21:03:51,M,3422.0,NaN,,comedian architecture enthusiast design nerd t...,NaN
2,1,0.0,NaN,10054.0,Family,Australia Remastered: Nature's Great Divide,NaN,NaN,The hard line that separates the natural world...,[The narrow strait between the Indonesian isla...,...,https://cdn.iview.abc.net.au/thumbs/i/do/DO195...,[['Host Aaron Pedersen']],"[ocean, informative, natural-world, animals, f...",2021-12-07 21:27:05,G,"[3240.0, 3240.0, 3200.0]",NaN,NaN,hard line separate natural world asia australi...,54.0
3,1,3.0,The Mule,24.0,Movies,The Mule,NaN,The Mule,A first-time drug mule tries desperately to pr...,A first-time drug mule tries desperately to pr...,...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW238...,"[['Director Tony Mahony', 'Angus Sampson']]","[abc1, film, drama, comedy-drama, drugs, organ...",2021-06-11 07:00:00,MA,5883.0,NaN,,firsttime drug mule try desperately prevent bo...,NaN
4,2,4.0,Play School: Acknowledgement of Country,193.0,Kids,Play School: Acknowledgement of Country,NaN,Play School: Acknowledgement of Country,"Join Luke, Miranda and Hunter to give an Ackno...","Join Luke, Miranda and Hunter to give an Ackno...",...,https://cdn.iview.abc.net.au/thumbs/i/ck/CK191...,[[]],[abc4kids],2021-07-08 06:00:00,G,1638.0,NaN,,join luke miranda hunter give acknowledgement ...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,30,5.0,Murray Whelan: Stiff,139.0,Movies,Murray Whelan: Stiff,NaN,Murray Whelan: Stiff,"Murray Whelan, a young political advisers, mus...","Murray Whelan, a young political advisers, mus...",...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW281...,[['Director John Clarke']],"[abc1, australia, drama, crime, crime-drama, 2...",2021-06-25 07:00:00,M,5567.0,NaN,,murray whelan young political adviser must pla...,NaN
62,31,6.0,Can We Save The Reef?,152.0,Family,Can We Save The Reef?,NaN,Can We Save The Reef?,An epic story of Australian and international ...,An epic story of Australian and international ...,...,https://cdn.iview.abc.net.au/thumbs/i/sc/SC160...,[[]],"[abc1, docs, nature, science, science-week, cl...",2018-05-12 14:58:55,G,3428.0,NaN,,epic story australian international scientist ...,NaN
63,31,12.0,Australia: The Wild Top End,155.0,Family,Australia: The Wild Top End,NaN,Australia: The Wild Top End,Come with us on an epic journey through some o...,Come with us on an epic journey through some o...,...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW284...,[[]],"[abc3, 3featured, family-viewing-abcme, family...",2022-03-16 09:45:00,PG,2602.0,NaN,,come u epic journey wildest landscape earth ta...,NaN
64,32,12.0,An Accidental Soldier,29.0,Movies,An Accidental Soldier,NaN,An Accidental Soldier,"On the Western Front in WWI, an Australian on ...","On the Western Front in WWI, an Australian on ...",...,https://cdn.iview.abc.net.au/thumbs/i/dr/DR120...,[['Director Rachel Ward']],"[abc1, drama, film, war, love, romance, adapta...",2021-06-11 07:00:00,M,5502.0,NaN,,western front wwi australian run find unexpect...,NaN


In [39]:
# recommended_films_div.dropna(subset=['id'], inplace=True)
# recommended_shows_div.dropna(subset=['id'], inplace=True)
recommended_mix_div = pd.concat([recommended_films_div, recommended_shows_div])
#choose the random 6 rows per each user_id
recommended_mix_div = recommended_mix_div.groupby('user_id').apply(lambda x: x.sample(2, replace=False))
recommended_mix_div.reset_index(drop=True, inplace=True)

In [40]:
recommended_mix_div.dropna(subset=['id'], inplace=True)

In [41]:
recommended_mix_div

,user_id,cluster,episode_title,id,category,title,series,episode_name,description,description2,...,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,features,index
5,2,6.0,Re-Frame,277.0,Documentary,Re-Frame 2021,NaN,Re-Frame 2021,"Showcasing the diversity of disability, Re-Fra...","Showcasing the diversity of disability, Re-Fra...",...,https://cdn.iview.abc.net.au/thumbs/i/rf/RF210...,[[]],"[abc1, abc2, docs, diversity, cald, disability...",2021-12-03 07:00:00,M,3493.0,NaN,,showcasing diversity disability reframe brings...,NaN
8,4,3.0,Capturing Cricket: Steve Waugh In India,232.0,Documentary,Capturing Cricket: Steve Waugh In India,NaN,Capturing Cricket: Steve Waugh In India,"Steve Waugh, the mastermind behind Australia's...","Steve Waugh, the mastermind behind Australia's...",...,https://cdn.iview.abc.net.au/thumbs/i/rf/RF192...,[['Host Steve Waugh']],"[abc1, australia, docs, factual, sport, divers...",2020-11-17 21:30:47,PG,3407.0,NaN,,steve waugh mastermind behind australia golden...,NaN
13,6,4.0,NaN,10572.0,Drama,Traces,NaN,NaN,"While attending an online forensic course, a y...",[While Kathy oversees the excavation of Marie'...,...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW296...,"[['Cast Molly Windsor', 'Jennifer Spence', 'La...","[crime-drama, gritty, mystery, suspense, thril...",2021-08-15 21:27:00,M,"[2618.0, 2592.0, 2712.0, 2642.0, 2581.0, 2593.0]",NaN,NaN,attending online forensic course young lab ass...,572.0
15,7,3.0,Australia Day Live,313.0,Comedy,Australia Day Live 2022,NaN,Australia Day Live 2022,A concert and fireworks spectacular from Sydne...,A concert and fireworks spectacular from Sydne...,...,https://cdn.iview.abc.net.au/thumbs/i/rv/RV210...,"[['Host Jeremy Fernandez', 'Casey Donovan', 'J...","[abc1, aussie, australia, event, concert, musi...",2022-01-26 21:31:00,PG,7871.0,NaN,,concert firework spectacular sydney featuring ...,NaN
19,9,3.0,Australia Day Live,313.0,Comedy,Australia Day Live 2022,NaN,Australia Day Live 2022,A concert and fireworks spectacular from Sydne...,A concert and fireworks spectacular from Sydne...,...,https://cdn.iview.abc.net.au/thumbs/i/rv/RV210...,"[['Host Jeremy Fernandez', 'Casey Donovan', 'J...","[abc1, aussie, australia, event, concert, musi...",2022-01-26 21:31:00,PG,7871.0,NaN,,concert firework spectacular sydney featuring ...,NaN
28,14,4.0,NaN,10572.0,Drama,Traces,NaN,NaN,"While attending an online forensic course, a y...",[While Kathy oversees the excavation of Marie'...,...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW296...,"[['Cast Molly Windsor', 'Jennifer Spence', 'La...","[crime-drama, gritty, mystery, suspense, thril...",2021-08-15 21:27:00,M,"[2618.0, 2592.0, 2712.0, 2642.0, 2581.0, 2593.0]",NaN,NaN,attending online forensic course young lab ass...,572.0
29,14,6.0,Re-Frame,277.0,Documentary,Re-Frame 2021,NaN,Re-Frame 2021,"Showcasing the diversity of disability, Re-Fra...","Showcasing the diversity of disability, Re-Fra...",...,https://cdn.iview.abc.net.au/thumbs/i/rf/RF210...,[[]],"[abc1, abc2, docs, diversity, cald, disability...",2021-12-03 07:00:00,M,3493.0,NaN,,showcasing diversity disability reframe brings...,NaN
31,15,3.0,Australia Day Live,313.0,Comedy,Australia Day Live 2022,NaN,Australia Day Live 2022,A concert and fireworks spectacular from Sydne...,A concert and fireworks spectacular from Sydne...,...,https://cdn.iview.abc.net.au/thumbs/i/rv/RV210...,"[['Host Jeremy Fernandez', 'Casey Donovan', 'J...","[abc1, aussie, australia, event, concert, musi...",2022-01-26 21:31:00,PG,7871.0,NaN,,concert firework spectacular sydney featuring ...,NaN
37,18,3.0,Australia Day Live,313.0,Comedy,Australia Day Live 2022,NaN,Australia Day Live 2022,A concert and fireworks spectacular from Sydne...,A concert and fireworks spectacular from Sydne...,...,https://cdn.iview.abc.net.au/thumbs/i/rv/RV210...,"[['Host Jeremy Fernandez', 'Casey Donovan', 'J...","[abc1, aussie, australia, event, concert, musi...",2022-01-26 21:31:00,PG,7871.0,NaN,,conce

In [ ]:
recommended_mix_austalia.to_csv(directory_path + 'RECOMMENDED_mix_austalia.csv', index=False)
recommended_mix_div.to_csv(directory_path + 'RECOMMENDED_mix_div.csv', index=False)